## Imports


In [ ]:
import fsspec
import xarray as xr
import s3fs
import distributed 

## Shared 

In [ ]:
# Start distributed client 

client = distributed.Client()
client 

In [ ]:


zenodo_zipped_url = "https://zenodo.org/records/8293998/files/code_and_data.zip"

# grab these from github repo
access_key_id = 
secret_access_key = 
endpoint_url = 'https://nyu1.osn.mghpcc.org'
write_fs = s3fs.S3FileSystem(
    key=access_key_id,
    secret=secret_access_key,
    client_kwargs={'endpoint_url': endpoint_url}
)


read_fs = fsspec.filesystem("zip", fo=zenodo_zipped_url)


## Raw Training Data

In [ ]:
training_data_raw_path = "code_and_data/Data/training_data_raw_time_index_49_149.nc"
raw_training_mapper = write_fs.get_mapper('leap-pangeo-manual/GOTM/raw_training_data.zarr')

raw_ds = xr.open_dataset(read_fs.open(training_data_raw_path), chunks={})
raw_ds.chunk({'indexes':2500000,'sigma': 5}).to_zarr(raw_training_mapper)


## Training Data SF

In [ ]:
training_data_SF_path = "code_and_data/Data/training_data_for_SF_hbl_gaps_filled.nc"
SF_training_mapper = write_fs.get_mapper('leap-pangeo-manual/GOTM/SF_training_data.zarr')

sf_ds = xr.open_dataset(read_fs.open(training_data_SF_path), chunks={})
sf_ds.chunk({'indexes':2500000,'sigma': 5}).to_zarr(SF_training_mapper)
